In [1]:
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
digit_input = Input(shape=(105,105,1))

x = Conv2D(64, (10, 10),activation='relu',input_shape=(105,105,1))(digit_input)
x = MaxPooling2D()(x)
x = Conv2D(128, (8, 8),activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, (6, 6),activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(256, (4, 4),activation='relu')(x)
out = Flatten()(x)
#out = Dense(4096,activation='sigmoid')(x)
vision_model = Model(digit_input,out)

letter1 = Input(shape=(105,105,1))
letter2 = Input(shape=(105,105,1))

out_1 = vision_model(letter1)
out_2 = vision_model(letter2)

def m_dist(A,B):
    return K.sum(K.abs(A-B),axis=1,keepdims=True)

merged_vector = Lambda(lambda x:m_dist(x[0],x[1]), output_shape=lambda inp_shp:(inp_shp[0][0],1))([out_1,out_2])

out_fin = Dense(4096,activation="sigmoid")(merged_vector)
final_output = Dense(1,activation="sigmoid")(out_fin)

final_model = keras.models.Model(inputs=[letter1, letter2], outputs=final_output)

In [12]:
from functools import partial 
def convolutional_mapper(my_input):
    # Convolutional Layer #1
        
        conv_relu = parital(tf.layers.conv2d,actication=tf.nn.relu)
        
        conv1 = conv_relu(inputs=input_layer,filters=64,kernel_size=[10, 10],padding="same")
        
        # Pooling Layer #1
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
        
        # Convolutional Layer #2 and Pooling Layer #2
        conv2 = conv_relu(inputs=pool1,filters=128,kernel_size=[8, 8],padding="same")

        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

        conv3 = tf.layers.conv2d(inputs=pool2,filters=128,kernel_size=[6, 6],padding="same",activation=tf.nn.relu)

        pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

        conv4 = tf.layers.conv2d(inputs=pool2,filters=128,kernel_size=[6, 6],padding="same",activation=tf.nn.relu)

        # Dense Layer
        pool5_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
        
        return pool5_flat

In [10]:
def compute_manhattan_distance(x, y):
    
    size_x = x.shape.dims[0]
    size_y = y.shape.dims[0]
    for i in range(size_x):
        tile_one = tf.reshape(tf.tile(x[i], [size_y]), [size_y, -1])
        eu_one = tf.expand_dims(tf.reduce_sum(tf.abs(tf.subtract(tile_one, y)), axis=1), axis=0)
        if i == 0:
            d = eu_one
        else:
            d = tf.concat([d, eu_one], axis=0)
            
    return d

In [11]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    
    # Input Layer
    
    input_layer1 = tf.reshape(features["image1"], [-1, 105, 105, 1])
    input_layer2 = tf.reshape(features["image2"], [-1, 105, 105, 1])
    
    
    with tf.variable_scope("mapped_image1"):
        mapped1 = convolutional_mapper(input_layer1)
        
    with tf.variable_scope("mapped_image2",reuse=True):
        mapped2 = convolutional_mapper(input_layer2)
        
    component_wise_distance = compute_manhattan_distance(mapped1,mapped2)
    
    dense_final = tf.layers.dense(inputs=component_wise_distance, units=4096, activation=tf.nn.sigmoid)
   
    logits = tf.layers.dense(inputs=dense_final, units=1)
    

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.sigmoid(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.binary_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

IndentationError: expected an indented block (<ipython-input-11-9f5b8bfb8595>, line 32)